# Time series analysis

In [2]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)


download("https://github.com/AllenDowney/ThinkStats/raw/v3/nb/thinkstats.py")

In [3]:
import numpy as np
import matplotlib.pyplot as plt

from thinkstats import decorate

A **time series** is a sequence of measurements from a system that
varies in time. One famous example is the "hockey stick graph" that
shows global average temperature over time (see
<https://en.wikipedia.org/wiki/Hockey_stick_graph>).

The example I work with in this chapter comes from Zachary M. Jones, a
researcher in political science who studies the black market for
cannabis in the U.S. (<http://zmjones.com/marijuana>). He collected data
from a web site called "Price of Weed" that crowdsources market
information by asking participants to report the price, quantity,
quality, and location of cannabis transactions
(<http://www.priceofweed.com/>). The goal of his project is to
investigate the effect of policy decisions, like legalization, on
markets. I find this project appealing because it is an example that
uses data to address important political questions, like drug policy.

I hope you will find this chapter interesting, but I'll take this
opportunity to reiterate the importance of maintaining a professional
attitude to data analysis. Whether and which drugs should be illegal are
important and difficult public policy questions; our decisions should be
informed by accurate data reported honestly.

## Importing and cleaning

The data I downloaded from Mr. Jones's site is in the repository for
this book. The following code reads it into a pandas `DataFrame`:

In [4]:
download("https://github.com/AllenDowney/ThinkStats/raw/v3/data/mj-clean.csv")

In [5]:
import pandas as pd

transactions = pd.read_csv("mj-clean.csv", parse_dates=[5])
transactions.head()

`parse_dates` tells `read_csv` to interpret values in column 5 as dates
and convert them to NumPy `datetime64` objects.

The `DataFrame` has a row for each reported transaction and the following
columns:

-   city: string city name.

-   state: two-letter state abbreviation.

-   price: price paid in dollars.

-   amount: quantity purchased in grams.

-   quality: high, medium, or low quality, as reported by the purchaser.

-   date: date of report, presumed to be shortly after date of purchase.

-   ppg: price per gram, in dollars.

-   state.name: string state name.

-   lat: approximate latitude of the transaction, based on city name.

-   lon: approximate longitude of the transaction.

Each transaction is an event in time, so we could treat this dataset as
a time series. But the events are not equally spaced in time; the number
of transactions reported each day varies from 0 to several hundred. Many
methods used to analyze time series require the measurements to be
equally spaced, or at least things are simpler if they are.

In order to demonstrate these methods, I divide the dataset into groups
by reported quality, and then transform each group into an equally
spaced series by computing the mean daily price per gram.

In [6]:
def group_by_quality_and_day(transactions):
    """Divides transactions by quality and computes mean daily price.

    transaction: DataFrame of transactions

    returns: map from quality to time series of ppg
    """
    groups = transactions.groupby("quality")
    dailies = {}
    for name, group in groups:
        dailies[name] = group_by_day(group)
    return dailies

`groupby` is a `DataFrame` method that returns a GroupBy object, `groups`;
used in a for loop, it iterates the names of the groups and the
`DataFrame`s that represent them. Since the values of `quality` are `low`,
`medium`, and `high`, we get three groups with those names.

The loop iterates through the groups and calls `GroupByDay`, which
computes the daily average price and returns a new `DataFrame`:

In [7]:
def group_by_day(transactions, func="mean"):
    """Groups transactions by day and compute the daily mean ppg.

    transactions: DataFrame of transactions

    returns: DataFrame of daily prices
    """
    grouped = transactions[["date", "ppg"]].groupby("date")
    daily = grouped.aggregate(func)
    daily["date"] = daily.index
    start = daily.date.iloc[0]
    one_year = np.timedelta64(52, "W")
    daily["years"] = (daily.date - start) / one_year
    return daily

The parameter, `transactions`, is a `DataFrame` that contains columns
`date` and `ppg`. We select these two columns, then group by `date`.

The result, `grouped`, is a map from each date to a `DataFrame` that
contains prices reported on that date. `aggregate` is a GroupBy method
that iterates through the groups and applies a function to each column
of the group; in this case there is only one column, `ppg`. So the
result of `aggregate` is a `DataFrame` with one row for each date and one
column, `ppg`.

Dates in these `DataFrame`s are stored as NumPy `datetime64` objects,
which are represented as 64-bit integers in nanoseconds. For some of the
analyses coming up, it will be convenient to work with time in more
human-friendly units, like years. So `GroupByDay` adds a column named
`date` by copying the `index`, then adds `years`, which contains the
number of years since the first transaction as a floating-point number.

The resulting `DataFrame` has columns `ppg`, `date`, and `years`.

In [8]:
dailies = group_by_quality_and_day(transactions)

## Plotting

The result from `GroupByQualityAndDay` is a map from each quality to a
`DataFrame` of daily prices. Here's the code I use to plot the three time
series:

In [10]:
import matplotlib.pyplot as plt

for i, (name, daily) in enumerate(dailies.items()):
    plt.subplot(3, 1, i + 1)
    title = "Price per gram ($)" if i == 0 else ""
    plt.scatter(daily.index, daily.ppg, s=10, label=name)
    decorate(ylim=[0, 20], title=title)

    if i == 2:
        plt.xticks(rotation=30)
    else:
        plt.xticks([])

The loop iterates through the
`DataFrame`s and creates a scatter plot for each. It is common to plot
time series with line segments between the points, but in this case
there are many data points and prices are highly variable, so adding
lines would not help.

Since the labels on the x-axis are dates, I use `plt.xticks` to
rotate the "ticks" 30 degrees, making them more readable.

One apparent feature in these
plots is a gap around November 2013. It's possible that data collection
was not active during this time, or the data might not be available. We
will consider ways to deal with this missing data later.

Visually, it looks like the price of high quality cannabis is declining
during this period, and the price of medium quality is increasing. The
price of low quality might also be increasing, but it is harder to tell,
since it seems to be more volatile. Keep in mind that quality data is
reported by volunteers, so trends over time might reflect changes in how
participants apply these labels.

## Linear regression

Although there are methods specific to time series analysis, for many
problems a simple way to get started is by applying general-purpose
tools like linear regression. The following function takes a `DataFrame`
of daily prices and computes a least squares fit, returning the model
and results objects from StatsModels:

In [11]:
import statsmodels.formula.api as smf


def run_linear_model(daily):
    model = smf.ols("ppg ~ years", data=daily)
    results = model.fit()
    return model, results

Then we can iterate through the qualities and fit a model to each:

In [12]:
from thinkstats import summarize_results

for name, daily in dailies.items():
    model, results = run_linear_model(daily)
    print(name)
    summarize_results(results)
    print()

The estimated slopes indicate that the price of high quality cannabis
dropped by about 71 cents per year during the observed interval; for
medium quality it increased by 28 cents per year, and for low quality it
increased by 57 cents per year. These estimates are all statistically
significant with very small p-values.

The $R^2$ value for high quality cannabis is 0.44, which means that time
as an explanatory variable accounts for 44% of the observed variability
in price. For the other qualities, the change in price is smaller, and
variability in prices is higher, so the values of $R^2$ are smaller (but
still statistically significant).

The following code plots the observed prices and the fitted values:

In [80]:
def plot_fitted_values(model, results, label=""):
    years = model.exog[:, 1]
    values = model.endog
    plt.scatter(years, values, s=15, label=label)
    plt.plot(years, results.fittedvalues, color="C1", label="model")
    decorate()

From `model`, `plot_fitted_values` gets `exog` and `endog` , NumPy
arrays with the exogenous (explanatory) and endogenous (dependent)
variables.
Then it makes a scatter plot of the data points and a line
plot of the fitted values.

The following figure shows the results for high quality cannabis.

In [81]:
model, results = run_linear_model(dailies["high"])
plot_fitted_values(model, results, label="high")

The model seems like a good linear fit for the data; nevertheless,
linear regression is not the most appropriate choice for this data:

-   First, there is no reason to expect the long-term trend to be a line
    or any other simple function. In general, prices are determined by
    supply and demand, both of which vary over time in unpredictable
    ways.

-   Second, the linear regression model gives equal weight to all data,
    recent and past. For purposes of prediction, we should probably give
    more weight to recent data.

-   Finally, one of the assumptions of linear regression is that the
    residuals are uncorrelated noise. With time series data, this
    assumption is often false because successive values are correlated.

The next section presents an alternative that is more appropriate for
time series data.

## Moving averages

Most time series analysis is based on the modeling assumption that the
observed series is the sum of three components:

-   Trend: A smooth function that captures persistent changes.

-   Seasonality: Periodic variation, possibly including daily, weekly,
    monthly, or yearly cycles.

-   Noise: Random variation around the long-term trend.

Regression is one way to extract the trend from a series, as we saw in
the previous section. But if the trend is not a simple function, a good
alternative is a **moving average**. A moving average divides the series
into overlapping regions, called **windows**, and computes the average
of the values in each window.

One of the simplest moving averages is the **rolling mean**, which
computes the mean of the values in each window. For example, if the
window size is 3, the rolling mean computes the mean of values 0 through
2, 1 through 3, 2 through 4, etc.

pandas provides `rolling_mean`, which takes a `Series` and a window size
and returns a new `Series`.

In [82]:
array = np.arange(10)

In [83]:
series = pd.Series(array)
series.rolling(3).mean()

The first two values are `nan`; the next value is the mean of the first
three elements, 0, 1, and 2. The next value is the mean of 1, 2, and 3.
And so on.

Before we can apply `rolling_mean` to the cannabis data, we have to deal
with missing values. There are a few days in the observed interval with
no reported transactions for one or more quality categories, and a
period in 2013 when data collection was not active.

In the `DataFrame`s we have used so far, these dates are absent; the index
skips days with no data. For the analysis that follows, we need to
represent this missing data explicitly. We can do that by "reindexing"
the `DataFrame`:

In [84]:
dates = pd.date_range(daily.index.min(), daily.index.max())
reindexed = daily.reindex(dates)

The first line computes a date range that includes every day from the
beginning to the end of the observed interval. The second line creates a
new `DataFrame` with all of the data from `daily`, but including rows for
all dates, filled with `nan`.

Now we can plot the rolling mean like this:

In [85]:
from thinkstats import underride


def scatter_series(xs, **options):
    """Makes a scatter plot from a Series.

    xs: Series
    options: options passed to scatter
    """
    options = underride(options, color="blue", alpha=0.2, s=30, edgecolors="none")
    ys = xs.values
    xs = xs.index
    plt.scatter(xs, ys, **options)

In [87]:
def plot_rolling_mean(daily, label):
    """Plots rolling mean.

    daily: DataFrame of daily prices
    """
    dates = pd.date_range(daily.index.min(), daily.index.max())
    reindexed = daily.reindex(dates)
    scatter_series(reindexed.ppg, s=15, alpha=0.2, label=label)
    roll_mean = pd.Series(reindexed.ppg).rolling(30).mean()
    plt.plot(roll_mean, label="rolling mean", color="#ff7f00")
    plt.xticks(rotation=30)
    decorate(ylabel="price per gram ($)")

In [88]:
plot_rolling_mean(daily, name)

The window size is 30, so each value in `roll_mean` is the mean of 30
values from `reindexed.ppg`.
The rolling mean
seems to do a good job of smoothing out the noise and extracting the
trend. The first 29 values are `nan`, and wherever there's a missing
value, it's followed by another 29 `nan`s. There are ways to fill in
these gaps, but they are a minor nuisance.

An alternative is the **exponentially-weighted moving average** (EWMA),
which has two advantages. First, as the name suggests, it computes a
weighted average where the most recent value has the highest weight and
the weights for previous values drop off exponentially. Second, the
pandas implementation of EWMA handles missing values better.

In [89]:
def plot_ewma(daily, label):
    """Plots rolling mean.

    daily: DataFrame of daily prices
    """
    dates = pd.date_range(daily.index.min(), daily.index.max())
    reindexed = daily.reindex(dates)
    scatter_series(reindexed.ppg, s=15, alpha=0.2, label=label)
    roll_mean = reindexed.ppg.ewm(30).mean()
    plt.plot(roll_mean, label="EWMA", color="#ff7f00")
    plt.xticks(rotation=30)
    decorate(ylabel="price per gram ($)")

The **span** parameter corresponds roughly to the window size of a
moving average; it controls how fast the weights drop off, so it
determines the number of points that make a non-negligible contribution
to each average.

The following figure shows the EWMA for the same data. 

In [90]:
plot_ewma(daily, name)

It is similar to the rolling mean, where they are both defined, but it has
no missing values, which makes it easier to work with. The values are
noisy at the beginning of the time series, because they are based on
fewer data points.

## Missing values

Now that we have characterized the trend of the time series, the next
step is to investigate seasonality, which is periodic behavior. Time
series data based on human behavior often exhibits daily, weekly,
monthly, or yearly cycles. In the next section I present methods to test
for seasonality, but they don't work well with missing data, so we have
to solve that problem first.

A simple and common way to fill missing data is to use a moving average.
The `Series` method `fillna` does just what we want:

In [91]:
from thinkstats import resample


def fill_missing(daily, span=30):
    """Fills missing values with an exponentially weighted moving average.

    Resulting DataFrame has new columns 'ewma' and 'resid'.

    daily: DataFrame of daily prices
    span: window size (sort of) passed to ewma

    returns: new DataFrame of daily prices
    """
    dates = pd.date_range(daily.index.min(), daily.index.max())
    reindexed = daily.reindex(dates)
    ewma = pd.Series(reindexed.ppg).ewm(span=span).mean()
    resid = (reindexed.ppg - ewma).dropna()
    fake_data = ewma + resample(resid, len(reindexed))
    reindexed["ppg"] = reindexed.ppg.fillna(fake_data)
    reindexed["ewma"] = ewma
    reindexed["resid"] = reindexed.ppg - ewma
    return reindexed

In [92]:
def plot_filled(daily, label):
    """Plots the EWMA and filled data.

    daily: DataFrame of daily prices
    """
    filled = fill_missing(daily, span=30)
    scatter_series(filled.ppg, s=15, alpha=0.2, label=label)
    plt.plot(filled.ewma, label="EWMA", color="#ff7f00")
    plt.xticks(rotation=30)
    decorate(ylabel="Price per gram ($)")

In [93]:
plot_filled(daily, name)

# TODO, what's up with that blip near the end?

Wherever `reindexed.ppg` is `nan`, `fillna` replaces it with the
corresponding value from `ewma`.

A drawback of this method is that it understates the noise in the
series. We can solve that problem by adding in resampled residuals:

In [94]:
dates = pd.date_range(daily.index.min(), daily.index.max())
reindexed = daily.reindex(dates)
ewma = pd.Series(reindexed.ppg).ewm(span=30).mean()

In [95]:
resid = (reindexed.ppg - ewma).dropna()
fake_data = ewma + resample(resid, len(reindexed))
reindexed["ppg"] = reindexed.ppg.fillna(ewma)

`resid` contains the residual values, not including days when `ppg` is
`nan`. `fake_data` contains the sum of the moving average and a random
sample of residuals. Finally, `fillna` replaces `nan` with values from
`fake_data`.

In [96]:
# TODO: missing figure in the sequence here?

The filled data is visually
similar to the actual values. Since the resampled residuals are random,
the results are different every time; later we'll see how to
characterize the error created by missing values.

## Serial correlation

As prices vary from day to day, you might expect to see patterns. If the
price is high on Monday, you might expect it to be high for a few more
days; and if it's low, you might expect it to stay low. A pattern like
this is called **serial correlation**, because each value is correlated
with the next one in the series.

To compute serial correlation, we can shift the time series by an
interval called a **lag**, and then compute the correlation of the
shifted series with the original:

In [97]:
from thinkstats import corr


def serial_corr(series, lag=1):
    xs = series[lag:]
    ys = series.shift(lag)[lag:]
    return corr(xs, ys)

After the shift, the first `lag` values are `nan`, so I use a slice to
remove them before computing `Corr`.

If we apply `SerialCorr` to the raw price data with lag 1, we find
serial correlation 0.48 for the high quality category, 0.16 for medium
and 0.10 for low. In any time series with a long-term trend, we expect
to see strong serial correlations; for example, if prices are falling,
we expect to see values above the mean in the first half of the series
and values below the mean in the second half.

It is more interesting to see if the correlation persists if you
subtract away the trend. For example, we can compute the residual of the
EWMA and then compute its serial correlation:

In [98]:
filled_dailies = {}
for name, daily in dailies.items():
    filled_dailies[name] = fill_missing(daily, span=30)

In [99]:
for name, filled in filled_dailies.items():
    r = serial_corr(filled.ppg, lag=1)
    print(name, r)

With lag=1, the serial correlations for the de-trended data are -0.022
for high quality, -0.015 for medium, and 0.036 for low. These values are
small, indicating that there is little or no one-day serial correlation
in this series.

To check for weekly, monthly, and yearly seasonality, I ran the analysis
again with different lags. Here are the results:

In [100]:
for name, filled in filled_dailies.items():
    r = serial_corr(filled.resid, lag=1)
    print(name, r)

In the next section we'll test whether these correlations are
statistically significant (they are not), but at this point we can
tentatively conclude that there are no substantial seasonal patterns in
these series, at least not with these lags.

## Autocorrelation

If you think a series might have some serial correlation, but you don't
know which lags to test, you can test them all! The **autocorrelation
function** is a function that maps from lag to the serial correlation
with the given lag. "Autocorrelation" is another name for serial
correlation, used more often when the lag is not 1.

StatsModels provides functions for time series
analysis, including `acf`, which computes the autocorrelation function:

In [101]:
import statsmodels.tsa.stattools as smtsa

filled = filled_dailies["high"]
acf = smtsa.acf(filled.resid, nlags=365, adjusted=True, fft=False)

`acf` computes serial correlations with lags from 0 through `nlags`. The
`unbiased` flag tells `acf` to correct the estimates for the sample
size. The result is an array of correlations. If we select daily prices
for high quality, and extract correlations for lags 1, 7, 30, and 365,
we can confirm that `acf` and `SerialCorr` yield approximately the same
results:

In [102]:
f"{acf[0]:.2} {acf[1]:.2} {acf[7]:.2} {acf[30]:.2} {acf[365]:.2}"

With `lag=0`, `acf` computes the correlation of the series with itself,
which is always 1.


The following figure shows autocorrelation functions for the
three quality categories, with `nlags=40`. 

In [107]:
def simulate_autocorrelation(daily, iters=1001, nlags=40):
    """Resample residuals, compute autocorrelation, and plot percentiles.

    daily: DataFrame
    iters: number of simulations to run
    nlags: maximum lags to compute autocorrelation
    """
    t = []
    for _ in range(iters):
        filled = fill_missing(daily, span=30)
        resid = resample(filled.resid)
        acf = smtsa.acf(resid, nlags=nlags, adjusted=True, fft=False)[1:]
        t.append(np.abs(acf))
    high = percentile_rows(t, [97.5])[0]
    low = -high
    lags = range(1, nlags + 1)
    plt.fill_between(lags, low, high, alpha=0.2, color="gray")

In [110]:
from thinkstats import percentile_rows


def plot_auto_correlation(dailies, nlags=40, add_weekly=False):
    """Plots autocorrelation functions.

    dailies: map from category name to DataFrame of daily prices
    nlags: number of lags to compute
    add_weekly: boolean, whether to add a simulated weekly pattern
    """
    daily = dailies["high"]
    simulate_autocorrelation(daily)
    for name, daily in dailies.items():
        if add_weekly:
            daily.ppg = add_weekly_seasonality(daily)
        filled = fill_missing(daily, span=30)
        acf = smtsa.acf(filled.resid, nlags=nlags, adjusted=True, fft=False)
        lags = np.arange(len(acf))
        plt.plot(lags[1:], acf[1:], label=name)

In [112]:
axis = [0, 41, -0.2, 0.2]
plot_auto_correlation(dailies, add_weekly=False)
decorate(xlabel="lag (day)", ylabel="correlation", loc="lower right")

The gray region shows the
normal variability we would expect if there is no actual
autocorrelation; anything that falls outside this range is statistically
significant, with a p-value less than 5%. Since the false positive rate
is 5%, and we are computing 120 correlations (40 lags for each of 3
times series), we expect to see about 6 points outside this region. In
fact, there are 7. We conclude that there are no autocorrelations in
these series that could not be explained by chance.

I computed the gray regions by resampling the residuals. You can see my
code in `timeseries.py`; the function is called
`SimulateAutocorrelation`.

To see what the autocorrelation function looks like when there is a
seasonal component, I generated simulated data by adding a weekly cycle.
Assuming that demand for cannabis is higher on weekends, we might expect
the price to be higher. To simulate this effect, I select dates that
fall on Friday or Saturday and add a random amount to the price, chosen
from a uniform distribution from \$0 to \$2.

In [113]:
def add_weekly_seasonality(daily):
    """Adds a weekly pattern.

    daily: DataFrame of daily prices

    returns: new DataFrame of daily prices
    """
    fri_or_sat = (daily.index.dayofweek == 4) | (daily.index.dayofweek == 5)
    fake = daily.ppg.copy()
    fake[fri_or_sat] += np.random.uniform(0, 2, fri_or_sat.sum())
    return fake

`frisat` is a boolean `Series`, `True` if the day of the week is Friday or
Saturday. `fake` is a new `DataFrame`, initially a copy of `daily`, which
we modify by adding random values to `ppg`. `frisat.sum()` is the total
number of Fridays and Saturdays, which is the number of random values we
have to generate.

The following figure shows autocorrelation functions for
prices with this simulated seasonality. 

In [116]:
axis = [0, 41, -0.2, 0.2]
plot_auto_correlation(dailies, add_weekly=True)
decorate(xlabel="lag (day)", ylabel="correlation", loc="lower right")

As expected, the correlations
are highest when the lag is a multiple of 7. For high and medium
quality, the new correlations are statistically significant. For low
quality they are not, because residuals in this category are large; the
effect would have to be bigger to be visible through the noise.

## Prediction

Time series analysis can be used to investigate, and sometimes explain,
the behavior of systems that vary in time. It can also make predictions.

The linear regressions we used in
Section [\[timeregress\]](#timeregress){reference-type="ref"
reference="timeregress"} can be used for prediction. The
RegressionResults class provides `predict`, which takes a `DataFrame`
containing the explanatory variables and returns a sequence of
predictions. Here's the code:

In [117]:
def generate_simple_prediction(results, years):
    """Generates a simple prediction.

    results: results object
    years: sequence of times (in years) to make predictions for

    returns: sequence of predicted values
    """
    n = len(years)
    inter = np.ones(n)
    d = dict(Intercept=inter, years=years, years2=years**2)
    predict_df = pd.DataFrame(d)
    predict = results.predict(predict_df)
    return predict

In [118]:
def plot_simple_prediction(name, results, years):
    predict = generate_simple_prediction(results, years)
    plt.scatter(daily.years, daily.ppg, alpha=0.2, label=name)
    plt.plot(years, predict, color="#ff7f00")
    xlim = years[0] - 0.1, years[-1] + 0.1
    decorate(
        title="Predictions",
        xlabel="Years",
        xlim=xlim,
        ylabel="Price per gram ($)",
        loc="upper right",
    )

In [119]:
name = "high"
daily = dailies[name]
_, results = run_linear_model(daily)
years = np.linspace(0, 5, 101)
plot_simple_prediction(name, results, years)

`results` is a RegressionResults object; `years` is the sequence of time
values we want predictions for. The function constructs a `DataFrame`,
passes it to `predict`, and returns the result.

If all we want is a single, best-guess prediction, we're done. But for
most purposes it is important to quantify error. In other words, we want
to know how accurate the prediction is likely to be.

There are three sources of error we should take into account:

-   Sampling error: The prediction is based on estimated parameters,
    which depend on random variation in the sample. If we run the
    experiment again, we expect the estimates to vary.

-   Random variation: Even if the estimated parameters are perfect, the
    observed data varies randomly around the long-term trend, and we
    expect this variation to continue in the future.

-   Modeling error: We have already seen evidence that the long-term
    trend is not linear, so predictions based on a linear model will
    eventually fail.

Another source of error to consider is unexpected future events.
Agricultural prices are affected by weather, and all prices are affected
by politics and law. As I write this, cannabis is legal in two states
and legal for medical purposes in 20 more. If more states legalize it,
the price is likely to go down. But if the federal government cracks
down, the price might go up.

Modeling errors and unexpected future events are hard to quantify.
Sampling error and random variation are easier to deal with, so we'll do
that first.

To quantify sampling error, I use resampling, as we did in
Section [\[regest\]](#regest){reference-type="ref" reference="regest"}.
As always, the goal is to use the actual observations to simulate what
would happen if we ran the experiment again. The simulations are based
on the assumption that the estimated parameters are correct, but the
random residuals could have been different. Here is a function that runs
the simulations:

In [120]:
def simulate_results(daily, iters=101, func=run_linear_model):
    """Run simulations based on resampling residuals.

    daily: DataFrame of daily prices
    iters: number of simulations
    func: function that fits a model to the data

    returns: list of result objects
    """
    _, results = func(daily)
    fake = daily.copy()
    result_seq = []
    for _ in range(iters):
        fake.ppg = results.fittedvalues + resample(results.resid)
        _, fake_results = func(fake)
        result_seq.append(fake_results)
    return result_seq

`daily` is a `DataFrame` containing the observed prices; `iters` is the
number of simulations to run.

`SimulateResults` uses `RunLinearModel`, from
Section [\[timeregress\]](#timeregress){reference-type="ref"
reference="timeregress"}, to estimate the slope and intercept of the
observed values.

Each time through the loop, it generates a "fake" dataset by resampling
the residuals and adding them to the fitted values. Then it runs a
linear model on the fake data and stores the RegressionResults object.

The next step is to use the simulated results to generate predictions:

In [121]:
def generate_predictions(result_seq, years, add_resid=False):
    """Generates an array of predicted values from a list of model results.

    When add_resid is False, predictions represent sampling error only.

    When add_resid is True, they also include residual error (which is
    more relevant to prediction).

    result_seq: list of model results
    years: sequence of times (in years) to make predictions for
    add_resid: boolean, whether to add in resampled residuals

    returns: sequence of predictions
    """
    n = len(years)
    d = dict(Intercept=np.ones(n), years=years, years2=years**2)
    predict_df = pd.DataFrame(d)
    predict_seq = []
    for fake_results in result_seq:
        predict = fake_results.predict(predict_df)
        if add_resid:
            predict += resample(fake_results.resid, n)
        predict_seq.append(predict)
    return predict_seq

`GeneratePredictions` takes the sequence of results from the previous
step, as well as `years`, which is a sequence of floats that specifies
the interval to generate predictions for, and `add_resid`, which
indicates whether it should add resampled residuals to the straight-line
prediction. `GeneratePredictions` iterates through the sequence of
RegressionResults and generates a sequence of predictions.


Finally, here's the code that plots a 90% confidence interval for the
predictions:

In [122]:
def plot_predictions(daily, years, iters=101, percent=90, func=run_linear_model):
    """Plots predictions.

    daily: DataFrame of daily prices
    years: sequence of times (in years) to make predictions for
    iters: number of simulations
    percent: what percentile range to show
    func: function that fits a model to the data
    """
    result_seq = simulate_results(daily, iters=iters, func=func)
    p = (100 - percent) / 2
    percents = p, 100 - p
    predict_seq = generate_predictions(result_seq, years, add_resid=True)
    low, high = percentile_rows(predict_seq, percents)
    plt.fill_between(years, low, high, alpha=0.3, color="gray")

    predict_seq = generate_predictions(result_seq, years, add_resid=False)
    low, high = percentile_rows(predict_seq, percents)
    plt.fill_between(years, low, high, alpha=0.5, color="gray")

`PlotPredictions` calls `GeneratePredictions` twice: once with
`add_resid=True` and again with `add_resid=False`. It uses
`PercentileRows` to select the 5th and 95th percentiles for each year,
then plots a gray region between these bounds.

The following figure shows the result.

In [123]:
years = np.linspace(0, 5, 101)
plt.scatter(daily.years, daily.ppg, alpha=0.1, label=name)
plot_predictions(daily, years)
xlim = years[0] - 0.1, years[-1] + 0.1
decorate(title="Predictions", xlabel="Years", xlim=xlim, ylabel="Price per gram ($)")

The dark gray region
represents a 90% confidence interval for the sampling error; that is,
uncertainty about the estimated slope and intercept due to sampling.

The lighter region shows a 90% confidence interval for prediction error,
which is the sum of sampling error and random variation.

These regions quantify sampling error and random variation, but not
modeling error. In general modeling error is hard to quantify, but in
this case we can address at least one source of error, unpredictable
external events.

The regression model is based on the assumption that the system is
**stationary**; that is, that the parameters of the model don't change
over time. Specifically, it assumes that the slope and intercept are
constant, as well as the distribution of residuals.

But looking at the moving averages in
[which figure?], it seems like the slope changes at least once
during the observed interval, and the variance of the residuals seems
bigger in the first half than the second.

As a result, the parameters we get depend on the interval we observe. To
see how much effect this has on the predictions, we can extend
`simulate_results` to use intervals of observation with different start
and end dates.

In [124]:
def simulate_intervals(daily, iters=101, func=run_linear_model):
    """Run simulations based on different subsets of the data.

    daily: DataFrame of daily prices
    iters: number of simulations
    func: function that fits a model to the data

    returns: list of result objects
    """
    result_seq = []
    starts = np.linspace(0, len(daily), iters).astype(int)
    for start in starts[:-2]:
        subset = daily[start:]
        _, results = func(subset)
        fake = subset.copy()
        for _ in range(iters):
            fake.ppg = results.fittedvalues + resample(results.resid)
            _, fake_results = func(fake)
            result_seq.append(fake_results)
    return result_seq

In [125]:
def plot_intervals(daily, years, iters=101, percent=90, func=run_linear_model):
    """Plots predictions based on different intervals.

    daily: DataFrame of daily prices
    years: sequence of times (in years) to make predictions for
    iters: number of simulations
    percent: what percentile range to show
    func: function that fits a model to the data
    """
    result_seq = simulate_intervals(daily, iters=iters, func=func)
    p = (100 - percent) / 2
    percents = p, 100 - p
    predict_seq = generate_predictions(result_seq, years, add_resid=True)
    low, high = percentile_rows(predict_seq, percents)
    plt.fill_between(years, low, high, alpha=0.2, color="gray")

The following figure shows the result for the medium quality
category.

In [126]:
name = "high"
daily = dailies[name]
plt.scatter(daily.years, daily.ppg, alpha=0.1, label=name)

plot_intervals(daily, years)
plot_predictions(daily, years)
xlim = years[0] - 0.1, years[-1] + 0.1
decorate(title="Predictions", xlabel="Years", xlim=xlim, ylabel="Price per gram ($)")

The lightest gray area shows a confidence interval that
includes uncertainty due to sampling error, random variation, and
variation in the interval of observation.

The model based on the entire interval has positive slope, indicating
that prices were increasing. But the most recent interval shows signs of
decreasing prices, so models based on the most recent data have negative
slope. As a result, the widest predictive interval includes the
possibility of decreasing prices over the next year.

## Further reading

Time series analysis is a big topic; this chapter has only scratched the
surface. An important tool for working with time series data is
autoregression, which I did not cover here, mostly because it turns out
not to be useful for the example data I worked with.

But once you have learned the material in this chapter, you are well
prepared to learn about autoregression. One resource I recommend is
Philipp Janert's book, *Data Analysis with Open Source Tools*, O'Reilly
Media, 2011. His chapter on time series analysis picks up where this one
leaves off.

## Glossary

-   **time series**: A dataset where each value is associated with a
    timestamp, often a series of measurements and the times they were
    collected.

-   **window**: A sequence of consecutive values in a time series, often
    used to compute a moving average.

-   **moving average**: One of several statistics intended to estimate
    the underlying trend in a time series by computing averages (of some
    kind) for a series of overlapping windows.

-   **rolling mean**: A moving average based on the mean value in each
    window.

-   **exponentially-weighted moving average (EWMA)**: A moving average
    based on a weighted mean that gives the highest weight to the most
    recent values, and exponentially decreasing weights to earlier
    values.

-   **span**: A parameter of EWMA that determines how quickly the
    weights decrease.

-   **serial correlation**: Correlation between a time series and a
    shifted or lagged version of itself.

-   **lag**: The size of the shift in a serial correlation or
    autocorrelation.

-   **autocorrelation**: A more general term for a serial correlation
    with any amount of lag.

-   **autocorrelation function**: A function that maps from lag to
    serial correlation.

-   **stationary**: A model is stationary if the parameters and the
    distribution of residuals does not change over time.

## Exercises

**Exercise:**   The linear model I used in this chapter has the obvious drawback that it is linear, and there is no reason to expect prices to change linearly over time. We can add flexibility to the model by adding a quadratic term, as we did in Section 11.3.

Use a quadratic model to fit the time series of daily prices, and use the model to generate predictions. You will have to write a version of `RunLinearModel` that runs that quadratic model, but after that you should be able to reuse code from the chapter to generate predictions.

In [127]:
def run_quadratic_model(daily):
    """Runs a linear model of prices versus years.

    daily: DataFrame of daily prices

    returns: model, results
    """
    daily["years2"] = daily.years**2
    model = smf.ols("ppg ~ years + years2", data=daily)
    results = model.fit()
    return model, results

In [128]:
name = "high"
daily = dailies[name]
model, results = run_quadratic_model(daily)
results.summary()

In [129]:
plot_fitted_values(model, results, label=name)
decorate(
    title="Fitted values", xlabel="Years", xlim=[-0.1, 3.8], ylabel="price per gram ($)"
)

In [131]:
years = np.linspace(0, 5, 101)
plt.scatter(daily.years, daily.ppg, alpha=0.1, label=name)
plot_predictions(daily, years, func=run_quadratic_model)
decorate(
    title="predictions",
    xlabel="Years",
    xlim=[years[0] - 0.1, years[-1] + 0.1],
    ylabel="Price per gram ($)",
)

**Exercise:** Write a definition for a class named `SerialCorrelationTest` that extends `HypothesisTest` from Section 9.2. It should take a series and a lag as data, compute the serial correlation of the series with the given lag, and then compute the p-value of the observed correlation.

Use this class to test whether the serial correlation in raw price data is statistically significant. Also test the residuals of the linear model and (if you did the previous exercise), the quadratic model.

In [132]:
from thinkstats import HypothesisTest


class SerialCorrelationTest(HypothesisTest):
    """Tests serial correlations by permutation."""

    def test_statistic(self, data):
        """Computes the test statistic.

        data: tuple of xs and ys
        """
        series, lag = data
        test_stat = abs(serial_corr(series, lag))
        return test_stat

    def run_model(self):
        """Run the model of the null hypothesis.

        returns: simulated data
        """
        series, lag = self.data
        permutation = series.reindex(np.random.permutation(series.index))
        return permutation, lag

In [133]:
name = "high"
daily = dailies[name]
series = daily.ppg
test = SerialCorrelationTest((series, 1))
pvalue = test.p_value()
print(test.actual, pvalue)

In [134]:
_, results = run_linear_model(daily)
series = results.resid
test = SerialCorrelationTest((series, 1))
pvalue = test.p_value()
print(test.actual, pvalue)

In [135]:
_, results = run_quadratic_model(daily)
series = results.resid
test = SerialCorrelationTest((series, 1))
pvalue = test.p_value()
print(test.actual, pvalue)

**Bonus Example:** There are several ways to extend the EWMA model to generate predictions. One of the simplest is something like this:

1. Compute the EWMA of the time series and use the last point as an intercept, `inter`.

2. Compute the EWMA of differences between successive elements in the time series and use the last point as a slope, `slope`.

3. To predict values at future times, compute `inter + slope * dt`, where `dt` is the difference between the time of the prediction and the time of the last observation.


In [136]:
name = "high"
daily = dailies[name]
filled = fill_missing(daily)
diffs = filled.ppg.diff()
plt.plot(diffs)
plt.xticks(rotation=30)
decorate(ylabel="Daily change in price per gram ($)")

In [137]:
filled["slope"] = diffs.ewm(span=365).mean()
plt.plot(filled.slope[-365:])
plt.xticks(rotation=30)
decorate(ylabel="EWMA of diff ($)")

In [138]:
start = filled.index[-1]
inter = filled.ewma.iloc[-1]
slope = filled.slope[-30:].mean()
start, inter, slope

In [139]:
dates = pd.date_range(filled.index.min(), filled.index.max() + np.timedelta64(365, "D"))
predicted = filled.reindex(dates)

In [140]:
predicted["date"] = predicted.index
one_day = np.timedelta64(1, "D")
predicted["days"] = (predicted.date - start) / one_day
predict = inter + slope * predicted.days
predicted["ewma"] = predicted.ewma.fillna(predict)

In [144]:
scatter_series(daily.ppg, alpha=0.1, label=name)
plt.plot(predicted.ewma, color="#ff7f00")
decorate()

As an exercise, run this analysis again for the other quality categories.